In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Preprocessing

### Importing Libraries

In [ ]:
!pip install chart_studio

     |████████████████████████████████| 71kB 2.3MB/s 


In [ ]:
#importing all libraries
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from nltk import word_tokenize
import os
import re
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go
import itertools
from scipy import stats
from ast import literal_eval

In [ ]:
!pip install nltk
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.stem.wordnet import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import gensim
import json


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
import keras.utils
from keras import utils as np_utils

#Keras Tokenizer just replaces certain punctuation characters and splits on the remaining space character.
#NLTK Tokenizer uses the Treebank tokenizer uses regular expressions to tokenize text as in Penn Treebank. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:

WINDOWS_SIZE = 10
labels=['none','mild','moderate','moderately severe', 'severe']
num_classes = len(labels)

### Creating a dataframe from the transcript file

In [ ]:
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            person_id = m.group(1)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            for i in range(0, lines):
                row = transcript.iloc[i]
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    if question != "":
                        rows_list.append(p)
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all.csv', sep=',')
    print("File was created")
    return all_participants

In [ ]:
#loading the data
data_path = "/content/drive/My Drive/transcripts/"
all_participants = transcripts_to_dataframe(data_path)

File was created


In [ ]:
all_participants.head(20)

,personId,question,answer
0,300,hi i'm ellie thanks for coming in today,
1,300,i was created to talk to people in a safe and ...,
2,300,think of me as a friend i don't judge i can't ...,
3,300,i'm here to learn about people and would love ...,
4,300,i'll ask a few questions to get us started and...,
5,300,how are you doing today,good
6,300,that's good,
7,300,where are you from originally,atlanta georgia
8,300,really,
9,300,why'd you move to l_a,um my parents are from here um


### Removing the stopwords and cleaning the data

In [ ]:
#Selecting the data from the data from based on the questions asked  ['where', 'when', 'how', 'why', 'are', 'what', 'do', 'have', 'can', 'did', 'is', 'could', 'so', 'tell', 'who', 'has']
interrogative = ["where", "when", "how","why","are","what","do","have","can","did","is", "could", "so", "tell", "who", "has"]
rslt_df = all_participants[all_participants.question.str.contains('|'.join(interrogative),na=False)]
rslt_df['answer'].replace('', np.nan, inplace=True)
rslt_df.dropna(subset = ["answer"], inplace=True)
rslt_df.reset_index(drop=True, inplace=True)
rslt_df
#rslt_df = rslt_df.to_csv("/content/drive/My Drive/rslt_df.csv")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,personId,question,answer
0,300,how are you doing today,good
1,300,where are you from originally,atlanta georgia
2,300,why'd you move to l_a,um my parents are from here um
3,300,how do you like l_a,i love it
4,300,what are some things you really like about l_a,i like the weather i like the opportunities u...
...,...,...,...
8698,390,that's so good to hear,mm
8699,390,is there anything you regret,um hm no um except meeting that one woman uh
8700,390,what advice would you give to yourself ten or ...,uh i don't know probably try a little harder ...
8701,390,tell me how you spend your ideal weekend,oh um getting out of town um going going away...


In [ ]:
#https://www.kaggle.com/currie32/the-importance-of-cleaning-text

def text_to_wordlist(text, remove_stopwords=True, stem_words=False):    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = stopwords.words("english")
        text = [wordnet_lemmatizer.lemmatize(w) for w in text if not w in stops ]
        text = [w for w in text if w != "nan" ]
    else:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\>", " ", text)
    
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [ ]:
#creating a corpus with the words from the answers without stopwords given by the patients
all_participants_mix = rslt_df.copy() # However, if you need the original list unchanged when the new list is modified, you can use copy() method. This is called shallow copy.
all_participants_mix['clean_answer'] = all_participants_mix.apply(lambda row: text_to_wordlist(row.answer).split(), axis=1)

In [ ]:
words = [w for w in all_participants_mix['clean_answer'].tolist()]
words = set(itertools.chain(*words)) #chain('ABC', 'DEF') --> A B C D E F
vocab_size = len(words)

### Getting the top common words used by the patients

In [ ]:
words

{'luggage',
 'reevaluate',
 'thoughtful',
 'dean',
 'northern',
 'intimate',
 'overjoyously',
 'molding',
 'chance',
 'lured',
 'correctly',
 'money',
 'obsessive',
 'invite',
 'category',
 'politely',
 'ecology',
 'got',
 'esteem',
 'practiced',
 'started',
 'permeable',
 'bed',
 'subtitle',
 'wholeso',
 'unsure',
 'worm',
 'common',
 'wellbutrin',
 'page',
 'authentic',
 'maintain',
 'celebrate',
 'infinite',
 'g',
 'bumped',
 'tree',
 'claim',
 'mysef',
 'deer',
 'issue',
 'har',
 'dunno',
 'provide',
 'slightly',
 'tendency',
 'longest',
 'let',
 'protecting',
 'diffferent',
 'messing',
 'sincere',
 'sentence',
 'violin',
 'harm',
 'quitter',
 'degree',
 'unprepared',
 'mass',
 'owned',
 'ge',
 'terrain',
 'prominent',
 'fascinated',
 'burbank',
 'limitation',
 'school',
 'prescribe',
 'abroad',
 'madagascar',
 'psychotherapy',
 'empathic',
 'meantime',
 'yawn',
 'toolbox',
 'conveyance',
 'chain',
 'defuse',
 'programmed',
 'soon',
 'sluggish',
 'sobbing',
 'near',
 'rage',
 'medi

# Tokenization

In [ ]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(all_participants_mix['clean_answer']) # fit_on_texts creates the vocabulary index based on word frequency.
#The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word 

tokenizer.fit_on_sequences(all_participants_mix['clean_answer']) #texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.

all_participants_mix['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix['clean_answer'])
all_participants_mix.head(15)

#   why are the output as numbers when text_to_sequences is called?
# the Tokenizer stores everything in the word_index during fit_on_texts. Then, when calling the texts_to_sequences method, only the top num_words are considered.

,personId,question,answer,clean_answer,t_answer
0,300,how are you doing today,good,[good],[14]
1,300,where are you from originally,atlanta georgia,"[atlanta, georgia]","[1809, 2053]"
2,300,why'd you move to l_a,um my parents are from here um,"[um, parent, um]","[1, 125, 1]"
3,300,how do you like l_a,i love it,[love],[64]
4,300,what are some things you really like about l_a,i like the weather i like the opportunities u...,"[like, weather, like, opportunity, um, yes]","[5, 132, 5, 327, 1, 41]"
5,300,how easy was it for you to get used to living ...,um it took a minute somewhat easy,"[um, took, minute, somewhat, easy]","[1, 164, 511, 581, 85]"
6,300,what are some things you don't really like abo...,congestion,[congestion],[1810]
7,300,what'd you study at school,um i took up business and administration,"[um, took, business, administration]","[1, 164, 181, 1246]"
8,300,are you still doing that,uh yeah i am here and there i'm on a break ri...,"[uh, yeah, i, am, break, right, plan, going, b...","[2, 19, 3, 6, 605, 46, 813, 38, 43, 2, 429, 1247]"
9,300,what's your dream job,uh probably to open up my own business,"[uh, probably, open, business]","[2, 27, 331, 181]"


In [ ]:
word_index = tokenizer.word_index
word_size = len(word_index)
print(word_index["sad"])


550


### Loading the train, validation and test data containing the PHQ Scores

In [ ]:
def load_avec_dataset_file(path,score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])  #cut function used to segregate array into bins 5 levels - 'none','mild','moderate','moderately severe', 'severe'
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = keras.utils.to_categorical(ds['level'], num_classes).tolist() #categorical levels 
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score','Gender']] 
    ds = ds.astype({"Participant_ID": float, "level": int, 'PHQ8_Score': int})
    return ds

In [ ]:
train = load_avec_dataset_file('/content/drive/My Drive/Depression_detect/train_split_Depression_AVEC2017 (1).csv','PHQ8_Score')
dev = load_avec_dataset_file('/content/drive/My Drive/Depression_detect/dev_split_Depression_AVEC2017.csv','PHQ8_Score')
test = load_avec_dataset_file('/content/drive/My Drive/Depression_detect/full_test_split.csv','PHQ8_Score')
print("Size: train= {}, dev= {}, test={}".format(len(train), len(dev), len(test)))
train.head()


Size: train= 107, dev= 35, test=47


,Participant_ID,level,cat_level,PHQ8_Score,Gender
0,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,0
1,304.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6,0
2,305.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7,1
3,310.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4,1
4,312.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2,1


In [ ]:
ds_total = pd.concat([dev,train,test])
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [ ]:
ds_total

,Participant_ID,level,cat_level,PHQ8_Score,Gender
0,302.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4,1
1,307.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4,0
2,331.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",8,1
3,335.0,3,"[0.0, 0.0, 0.0, 1.0, 0.0]",12,0
4,346.0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]",23,0
...,...,...,...,...,...
42,467.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,1
43,469.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",3,0
44,470.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",3,0
45,480.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,1


In [ ]:
ds_total.to_csv('/content/drive/My Drive/Depression_detect/ds_total.csv', sep='\t')
print("File was created")

File was created


### Splitting the dataset and grouping them based on the 5 different level

In [ ]:
def split_by_phq_level(ds):
    none_ds = ds[ds['level']==0]
    mild_ds = ds[ds['level']==1]
    moderate_ds = ds[ds['level']==2]
    moderate_severe_ds = ds[ds['level']==3]
    severe_ds = ds[ds['level']==4]
    return (none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds)

In [ ]:
none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_total)
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(none_ds), len(mild_ds), len(moderate_ds), len(moderate_severe_ds), len(severe_ds)))

Quantity per none_ds: 26, mild_ds: 70, moderate_ds 47, moderate_severe_ds: 24, severe_ds 22


### Merging the phrases_lp (consisting of the transcripts) and ds_total consisting of PHQ Score

In [ ]:
ds_lp = ds_total.merge(all_participants_mix,left_on=ds_total.Participant_ID.astype(int), right_on=all_participants_mix.personId.astype(int))
ds_lp.drop(ds_lp[ds_lp["t_answer"].map(len) < 3].index, inplace = True)
ds_lp.drop(['key_0','Participant_ID','cat_level','PHQ8_Score',	'Gender',	'personId',	'question'],axis=1)

,level,answer,clean_answer,t_answer
0,1,i'm fine how about yourself,"[i, am, fine]","[3, 6, 266]"
1,1,i'm from los angeles california,"[i, am, los, angeles, california]","[3, 6, 216, 221, 232]"
3,1,um all my family's here friends a mixture of ...,"[um, family, friend, mixture, people, lot, thing]","[1, 59, 34, 2090, 13, 18, 8]"
4,1,early childhood education,"[early, childhood, education]","[463, 899, 370]"
5,1,no not right now but i would love to get back...,"[right, would, love, get, back]","[46, 10, 64, 17, 43]"
...,...,...,...,...
8697,2,i thought you weren't a therapist that's a ty...,"[thought, therapist, that, typical, therapist,...","[100, 352, 20, 2032, 352, 186, 46, 9]"
8698,2,um yesterday i was watching a movie with my g...,"[um, yesterday, watching, movie, girlfriend, l...","[1, 369, 474, 202, 252, 883, 5293, 97]"
8699,2,um very honest um sometimes brutally so but i...,"[um, honest, um, sometimes, brutally, i, am, g...","[1, 388, 1, 44, 3338, 3, 6, 14, 281, 12, 34, 1..."
8700,2,um i guess well it's what i'm working on whic...,"[um, guess, well, i, am, working, trying, impr...","[1, 35, 15, 3, 6, 116, 96, 964, 1405, 57, 53, ..."


In [ ]:
ds_lp.columns

Index(['key_0', 'Participant_ID', 'level', 'cat_level', 'PHQ8_Score', 'Gender',
       'personId', 'question', 'answer', 'clean_answer', 't_answer'],
      dtype='object')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ds_lp['clean_answer'], ds_lp['level'], test_size=0.20, random_state=42)

### TF-IDF classifier

In [ ]:
def identity_tokenizer(text):
    return text

# Initalise the TfIdf vectoriser 
tvec = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)
# Fit the training data on the model
tvec.fit(X_train)

# Transform training data into sparse matrix
X_train_tvec = tvec.transform(X_train)
# Transform training data into sparse matrix
X_test_tvec = tvec.transform(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



## SVM Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
lr = LogisticRegression(random_state=1)

tvec_score = cross_val_score(lr, X_train_tvec, y_train, cv=3)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
print('Tfidf Vectorizer Score:', tvec_score.mean())

Tfidf Vectorizer Score: 0.38255189659522487


In [ ]:
predictions = baseline_model.predict(x_test)

In [ ]:
accuracy_score(y_test, predictions)

0.3738225086762519

In [ ]:
print(classification_report(y_test, predictions, digits=5))

              precision    recall  f1-score   support

           0    0.36585   0.04505   0.08021       333
           1    0.38149   0.85857   0.52826       898
           2    0.35762   0.25392   0.29698       638
           3    0.33333   0.00272   0.00539       368
           4    0.27273   0.01027   0.01980       292

    accuracy                        0.37643      2529
   macro avg    0.34220   0.23411   0.18613      2529
weighted avg    0.35384   0.37643   0.27613      2529



# GloVe + Naive Bayes Model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ds_lp['answer'], ds_lp['level'], test_size=0.3, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

def identity_tokenizer(text):
    return text

nb = Pipeline([('vect', CountVectorizer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
y_pred = nb.predict(x_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.35746157659890926
              precision    recall  f1-score   support

           0    0.22857   0.05755   0.09195       278
           1    0.38192   0.74011   0.50385       708
           2    0.31106   0.29563   0.30315       504
           3    0.33333   0.07116   0.11728       267
           4    0.33333   0.05000   0.08696       260

    accuracy                        0.35746      2017
   macro avg    0.31765   0.24289   0.22064      2017
weighted avg    0.33039   0.35746   0.29202      2017



## GloVe + Logistric Regression

In [ ]:
phrases_lp = pd.read_csv('/content/drive/My Drive/Final-year-project-Phase 2/phrases_lp.csv', sep='\t', converters={"t_answer": literal_eval}) 
phrases_lp.head(15)


,Unnamed: 0,personId,answer,t_answer
0,0,300,"['good', 'atlanta', 'georgia', 'um', 'parent',...","[16, 1634, 1997, 1, 131, 1, 63, 5, 142, 5]"
1,1,300,"['atlanta', 'georgia', 'um', 'parent', 'um', '...","[1634, 1997, 1, 131, 1, 63, 5, 142, 5, 334]"
2,2,300,"['georgia', 'um', 'parent', 'um', 'love', 'lik...","[1997, 1, 131, 1, 63, 5, 142, 5, 334, 1]"
3,3,300,"['um', 'parent', 'um', 'love', 'like', 'weathe...","[1, 131, 1, 63, 5, 142, 5, 334, 1, 39]"
4,4,300,"['parent', 'um', 'love', 'like', 'weather', 'l...","[131, 1, 63, 5, 142, 5, 334, 1, 39, 1]"
5,5,300,"['um', 'love', 'like', 'weather', 'like', 'opp...","[1, 63, 5, 142, 5, 334, 1, 39, 1, 154]"
6,6,300,"['love', 'like', 'weather', 'like', 'opportuni...","[63, 5, 142, 5, 334, 1, 39, 1, 154, 527]"
7,7,300,"['like', 'weather', 'like', 'opportunity', 'um...","[5, 142, 5, 334, 1, 39, 1, 154, 527, 608]"
8,8,300,"['weather', 'like', 'opportunity', 'um', 'yes'...","[142, 5, 334, 1, 39, 1, 154, 527, 608, 100]"
9,9,300,"['like', 'opportunity', 'um', 'yes', 'um', 'to...","[5, 334, 1, 39, 1, 154, 527, 608, 100, 1998]"


## Random Sampling of dataset

In [ ]:
b_none_ds = ds_total[ds_total['level']==0]
b_mild_ds = ds_total[ds_total['level']==1].sample(26)
b_moderate_ds = ds_total[ds_total['level']==2].sample(26)
b_moderate_severe_ds = ds_total[ds_total['level']==3]
b_severe_ds = ds_total[ds_total['level']==4]

ds_total_b = pd.concat([b_none_ds, b_mild_ds, b_moderate_ds, b_moderate_severe_ds, b_severe_ds])

In [ ]:
ds_lp = pd.merge(ds_total, phrases_lp,left_on='Participant_ID', right_on='personId')
ds_lp.drop(ds_lp[ds_lp["t_answer"].map(len) < 10].index, inplace = True)
ds_lp_b = pd.merge(ds_total_b, phrases_lp,left_on='Participant_ID', right_on='personId')
ds_lp_b.drop(ds_lp_b[ds_lp_b["t_answer"].map(len) < 10].index, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds_lp['answer'], ds_lp['level'], test_size=0.20, random_state=42)

## Loading the Glove Embeddings into a file and putting each vector into an np array

In [ ]:
embeddings_index = dict()
f = open('/content/drive/My Drive/glove.6B.100d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
       coefs = np.asarray(values[1:], dtype='float32')
       embeddings_index[word] = coefs
    except ValueError:
       pass
f.close()

400000it [00:12, 32573.12it/s]


In [ ]:
stop_words = stopwords.words('english')
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

xtrain_glove = [sent2vec(x) for x in tqdm(X_train)]
xtest_glove = [sent2vec(x) for x in tqdm(X_test)]

print('Checkpoint2 -Normalized Vector for Sentences are created')

xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)



100%|██████████| 112909/112909 [00:36<00:00, 3072.71it/s]

100%|██████████| 28228/28228 [00:08<00:00, 3398.24it/s]


Checkpoint2 -Normalized Vector for Sentences are created


In [ ]:
train_target = train
classifier = LogisticRegression(solver='lbfgs')
classifier.fit(xtrain_glove, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print('Training LogisticRegression Classifier is complete!!')
predictions = classifier.predict(xtest_glove)

Training LogisticRegression Classifier is complete!!


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4398
           1       0.34      1.00      0.51      9688
           2       0.00      0.00      0.00      7575
           3       0.00      0.00      0.00      3587
           4       0.00      0.00      0.00      2980

    accuracy                           0.34     28228
   macro avg       0.07      0.20      0.10     28228
weighted avg       0.12      0.34      0.18     28228



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

